In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

2024-03-07 14:30:00.900045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [3]:
prompt = "\n"

In [4]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [9]:
from torch.utils.data import DataLoader
from datasets import load_dataset
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1")
train_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")
eval_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="validation")
test_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="test")
                

In [31]:
batch_size = 12
#test_dataset_text = [test_dataset[i : i + batch_size]["text"] for i in range(0, len(test_dataset), batch_size)]
#def batch_iterator():
#    for i in range(0, len(dataset), batch_size):
#        yield dataset[i : i + batch_size]["text"]

In [8]:
# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
embedding_size

50257

In [14]:
column_names = dataset["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)
block_size = tokenizer.model_max_length

Running tokenizer on dataset:   0%|          | 0/4358 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/36718 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [41]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    desc=f"Grouping texts in chunks of {block_size}",
)

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]
test_dataset = lm_datasets["test"]

In [47]:
import random
for index in random.sample(range(len(eval_dataset)), 1):
    print(tokenizer.decode(eval_dataset[index]['input_ids'])[:100])
    print(tokenizer.decode(eval_dataset[index]['labels'])[:100])
    #print(f"Sample {index} of the training set: {train_dataset[index]}.")

 Press Association named him Co @-@ Driver of the Year with Earnhardt after Richmond accumulated 13 
 Press Association named him Co @-@ Driver of the Year with Earnhardt after Richmond accumulated 13 


In [45]:
# DataLoaders creation:
from transformers import default_data_collator

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=batch_size
)
test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=batch_size
)

In [37]:
import torch
for epoch in range(0, 1):
    active_dataloader = eval_dataloader
    model.eval()
    losses = []
    for step, batch in enumerate(active_dataloader):
        print(step)
        print(batch)
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))
        print(loss)

    losses = torch.cat(losses)
    try:
        eval_loss = torch.mean(losses)
        perplexity = math.exp(eval_loss)
    except OverflowError:
        perplexity = float("inf")

    logger.info(f"epoch {epoch}: perplexity: {perplexity} eval_loss: {eval_loss}")

    print({
            "perplexity": perplexity,
            "eval_loss": eval_loss,
            "epoch": epoch,
            "step": completed_steps,
        },
        step=completed_steps,
    )

0
{'input_ids': tensor([[  796,  8074, 20272,  ...,   309,   893,    69],
        [   73,   585,   268,  ...,   416,   262,  7318],
        [  290, 19717, 12275,  ...,   290,   257,  4957],
        ...,
        [   31, 12877, 16051,  ..., 16570,   837,   547],
        [ 9118,   287,   262,  ...,   400, 12761,   764],
        [  220,   198,   362,  ...,   837,  7997,   416]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  796,  8074, 20272,  ...,   309,   893,    69],
        [   73,   585,   268,  ...,   416,   262,  7318],
        [  290, 19717, 12275,  ...,   290,   257,  4957],
        ...,
        [   31, 12877, 16051,  ..., 16570,   837,   547],
        [ 9118,   287,   262,  ...,   400, 12761,   764],
        [  220,   198,   362,  ...,   837,  7997,   416]])}


KeyboardInterrupt: 

In [49]:
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    LlamaConfig,
    default_data_collator,
)
  
# Create DataLoaders for the training and validation dataset
train_dataloader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=train_config.batch_size_training,
    num_workers=train_config.num_workers_dataloader,
    pin_memory=True,
    sampler=train_sampler if train_sampler else None,
    drop_last=True,
    collate_fn=default_data_collator,
)

for i, batch in enumerate(train_dataloader):
    print(batch)
    break

NameError: name 'dataset_train' is not defined